# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("weather.csv")
weather_data.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Provideniya,64.3833,-173.3000,11.73,85,99,3.27,RU,1628244960
1,1,Petropavlovsk-Kamchatskiy,53.0452,158.6483,12.33,62,75,3.00,RU,1628244960
2,2,Nikolskoye,59.7035,30.7861,20.94,66,55,4.96,RU,1628244960
3,3,Kibaya,-5.3000,36.5667,24.18,37,21,3.15,TZ,1628244961
4,4,Saskylakh,71.9167,114.0833,15.10,47,100,6.24,RU,1628244961


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

# Store Lat and Lng into locations 
locations = weather_data[["Lat", "Lng"]]
# Store Humudity 
humidity = weather_data["Humidity"]

In [4]:
# Plot the Heatmap 
fig = gmaps.figure(center=[0, 0], zoom_level = 2)

# Create Heat layer 
heat_layer= gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer 
fig.add_layer(heat_layer)

# Display figure 
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with max temp lower than 80, but higher than 70.
# Wind speed less than 10 mph, zero cloudiness. 

city_weather_df = weather_data.loc[(weather_data["Max Temp"] > 70) & (weather_data["Max Temp"] < 80) & \
                                  (weather_data["Wind Speed"] < 10) & weather_data["Cloudiness"] == 0].dropna()
city_weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Provideniya,64.3833,-173.3000,11.73,85,99,3.27,RU,1628244960
1,1,Petropavlovsk-Kamchatskiy,53.0452,158.6483,12.33,62,75,3.00,RU,1628244960
2,2,Nikolskoye,59.7035,30.7861,20.94,66,55,4.96,RU,1628244960
3,3,Kibaya,-5.3000,36.5667,24.18,37,21,3.15,TZ,1628244961
4,4,Saskylakh,71.9167,114.0833,15.10,47,100,6.24,RU,1628244961


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable hotel_df
hotel_df = city_weather_df.reset_index(drop=True)
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,Provideniya,64.3833,-173.3000,11.73,85,99,3.27,RU,1628244960,
1,1,Petropavlovsk-Kamchatskiy,53.0452,158.6483,12.33,62,75,3.00,RU,1628244960,
2,2,Nikolskoye,59.7035,30.7861,20.94,66,55,4.96,RU,1628244960,
3,3,Kibaya,-5.3000,36.5667,24.18,37,21,3.15,TZ,1628244961,
4,4,Saskylakh,71.9167,114.0833,15.10,47,100,6.24,RU,1628244961,


In [7]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius" : 5000,
          "type": "hotel",
         "keyword": "hotels",
         "radius": 5000, 
         "key": g_key}

# Lat/Lng for loop
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # Add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    #API request 
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    #Extract Results 
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]["name"]}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
    
    print("----------------------------")
    
    # Set time.sleep()
    time.sleep(1)
    
    #End of search result
    print("----------Search Complete-------")
    

Retrieving Results for Index 0: Provideniya.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 1: Petropavlovsk-Kamchatskiy.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 2: Nikolskoye.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 3: Kibaya.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 4: Saskylakh.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 5: Beira.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 6: Lagoa.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Ind

----------Search Complete-------
Retrieving Results for Index 60: Hohoe.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 61: Hamilton.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 62: Xai-Xai.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 63: Ribeira Grande.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 64: Barrow.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 65: Norfolk.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 66: Hobart.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retr

----------Search Complete-------
Retrieving Results for Index 119: Rome.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 120: Richards Bay.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 121: Harper.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 122: Qaanaaq.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 123: Lemon Tree Passage.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 124: Stornoway.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 125: Pelym.
Missing field/result...skipping.
----------------------------
----------Search Compl

----------Search Complete-------
Retrieving Results for Index 178: Santa Cruz de la Sierra.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 179: Tautira.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 180: Coquimbo.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 181: Smoky Lake.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 182: Mokolo.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 183: San Jose.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 184: São João da Barra.
Missing field/result...skipping.
----------------------------
----

----------Search Complete-------
Retrieving Results for Index 237: Mahébourg.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 238: Cayenne.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 239: Korsholm.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 240: Eyl.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 241: Nikel.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 242: Forsytheganj.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 243: Tasiilaq.
Missing field/result...skipping.
----------------------------
----------Search Complete------

----------Search Complete-------
Retrieving Results for Index 296: Gladstone.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 297: Río Gallegos.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 298: Gamba.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 299: Samarai.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 300: Harindānga.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 301: Thessalon.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 302: Nhlangano.
Missing field/result...skipping.
----------------------------
----------Search Compl

----------Search Complete-------
Retrieving Results for Index 355: Haradok.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 356: Isangel.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 357: Lompoc.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 358: Belyy Yar.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 359: Katobu.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 360: Dali.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 361: Shahrud.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retr

----------Search Complete-------
Retrieving Results for Index 415: Talnakh.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 416: Tumen.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 417: Sinnamary.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 418: Beloha.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 419: Tongchuanshi.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 420: Bundaberg.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 421: Pacifica.
Missing field/result...skipping.
----------------------------
----------Search Complete--

----------Search Complete-------
Retrieving Results for Index 474: Abilene.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 475: Pochutla.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 476: Kumul.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 477: Haines Junction.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 478: Swan Hill.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 479: Karonga.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 480: Ampanihy.
Missing field/result...skipping.
----------------------------
----------Search Complet

----------Search Complete-------
Retrieving Results for Index 534: Nishihara.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 535: Atar.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 536: Tākestān.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 537: Kitob.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 538: Néa Anchiálos.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 539: Mercedes.
Missing field/result...skipping.
----------------------------
----------Search Complete-------
Retrieving Results for Index 540: Umluj.
Missing field/result...skipping.
----------------------------
----------Search Complete------

In [8]:
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,Provideniya,64.3833,-173.3000,11.73,85,99,3.27,RU,1628244960,
1,1,Petropavlovsk-Kamchatskiy,53.0452,158.6483,12.33,62,75,3.00,RU,1628244960,
2,2,Nikolskoye,59.7035,30.7861,20.94,66,55,4.96,RU,1628244960,
3,3,Kibaya,-5.3000,36.5667,24.18,37,21,3.15,TZ,1628244961,
4,4,Saskylakh,71.9167,114.0833,15.10,47,100,6.24,RU,1628244961,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))